## 提取資料path和label到array

In [1]:
import cv2
import numpy as np


In [2]:

y_train=[]
train_paths=[]


# Read text file containing image paths
with open('train.txt', 'r') as f:
    for line in f.readlines():  
    # print(line[:-1].split(' '))
        image_train = line[:-1].split(' ')

        train_paths.append(str(image_train[0]))
        y_train.append(image_train[1])



In [3]:
y_test=[]
test_paths=[]

# Read text file containing image paths
with open('test.txt', 'r') as f:
    from tqdm.notebook import tqdm

    for line in f.readlines():  
        # print(line[:-1].split(' '))
        image_test = line[:-1].split(' ') 

        test_paths.append(image_test[0])
        y_test.append(image_test[1])

In [4]:
y_val=[]
val_paths=[]

# Read text file containing image paths
with open('val.txt', 'r') as f:
    from tqdm.notebook import tqdm

    for line in f.readlines():  
        # print(line[:-1].split(' '))
        image_val = line[:-1].split(' ') 

        val_paths.append(image_val[0])
        y_val.append(image_val[1])

## 提取feature

* 用於Lenet5

In [1]:
def get_training_data(path, featureList):

    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (32, 32))
    img = img.astype(np.float32).reshape(32, 32, 1) / 255.0
    featureList.append(img)

* HOG用於perceptron

In [2]:
from skimage.feature import hog

# 設定 HOG 參數
orientations = 9
pixels_per_cell = (32, 32)
cells_per_block = (3, 3)
#設定圖片大小
IMG_WIDTH = 128
IMG_HEIGHT = 128

def extract_histogram(image_path):
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block)

    return hog_features

In [11]:
from tqdm.notebook import tqdm

In [12]:
%%time
#訓練集

x_train=[] #hog
x_train_lenet=[] #lenet5

with tqdm(total=len(train_paths)) as pbar:
    for path in train_paths:
        try:
            #hog
            features = extract_histogram(path)
            x_train.append(features)
            #lenet5
            get_training_data(path, x_train_lenet)

        except Exception as e:
            pass
        finally:
            pbar.update()

  0%|          | 0/63325 [00:00<?, ?it/s]

CPU times: user 34.4 s, sys: 2.83 s, total: 37.2 s
Wall time: 41.6 s


In [13]:
%%time
#測試集

x_test=[] #hog
x_test_lenet=[] #lenet

with tqdm(total=len(test_paths)) as pbar:
    for path in test_paths:
        try:
            #hog
            features = extract_histogram(path)
            x_test.append(features)
            #lenet
            get_training_data(path, x_test_lenet)

        except Exception as e:
            pass
        finally:
            pbar.update()

  0%|          | 0/450 [00:00<?, ?it/s]

CPU times: user 262 ms, sys: 20.6 ms, total: 283 ms
Wall time: 280 ms


In [14]:
%%time
#驗證集

x_val=[] #hog
x_val_lenet=[] #lenet

with tqdm(total=len(val_paths)) as pbar:
    for path in val_paths:
        try:
            #hog
            features = extract_histogram(path)
            x_val.append(features)
            #lenet
            get_training_data(path, x_val_lenet)

        except Exception as e:
            pass
        finally:
            pbar.update()

  0%|          | 0/450 [00:00<?, ?it/s]

CPU times: user 296 ms, sys: 27.1 ms, total: 323 ms
Wall time: 365 ms


## 特徵儲存

* feature用於lenet5

In [15]:
x_test_lenet=np.array(x_test_lenet)
x_val_lenet=np.array(x_val_lenet)
x_train_lenet=np.array(x_train_lenet)

print(x_test_lenet.shape)
print(x_val_lenet.shape)
print(x_train_lenet.shape)

(450, 32, 32, 1)
(450, 32, 32, 1)
(63325, 32, 32, 1)


In [16]:
np.save('feature/train_features_lenet',x_train_lenet)
np.save('feature/val_features_lenet',x_val_lenet)
np.save('feature/test_features_lenet',x_test_lenet)

* feature用於hog

In [17]:
x_train_np = np.array(x_train, dtype=object)
x_test_np = np.array(x_test, dtype=object)
x_val_np = np.array(x_val, dtype=object)

In [18]:
np.save('feature/train_features',x_train_np)
np.save('feature/val_features',x_val_np)
np.save('feature/test_features',x_test_np)


* label儲存

In [19]:
np.save('feature/train_labels',y_train)
np.save('feature/val_labels',y_val)
np.save('feature/test_labels',y_test)